In [1]:
"""
Trialwise Decoding
==================

In this example, we will use a convolutional neural network on the
[Physiobank EEG Motor Movement/Imagery Dataset](https://www.physionet.org/physiobank/database/eegmmidb/) to decode two classes:

1. Executed and imagined opening and closing of both hands
2. Executed and imagined opening and closing of both feet

<div class="alert alert-warning">

We use only one subject (with 90 trials) in this tutorial for demonstration
purposes. A more interesting decoding task with many more trials would be
to do cross-subject decoding on the same dataset.

</div>
"""

# Enable logging

import sys
import logging
import importlib
importlib.reload(logging)  # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                    level=logging.INFO, stream=sys.stdout)


##############################################################################
# Load data
# ---------

# You can load and preprocess your EEG dataset in any way,
# Braindecode only expects a 3darray (trials, channels, timesteps) of input
# signals `X` and a vector of labels `y` later (see below). In this tutorial,
# we will use the [MNE](https://www.martinos.org/mne/stable/index.html)
# library to load an EEG motor imagery/motor execution dataset. For a
# tutorial from MNE using Common Spatial Patterns to decode this data, see
# [here](http://martinos.org/mne/stable/auto_examples/decoding/plot_decoding_csp_eeg.html).
# For another library useful for loading EEG data, take a look at
# [Neo IO](https://pythonhosted.org/neo/io.html).

import mne
from mne.io import concatenate_raws

# 5,6,7,10,13,14 are codes for executed and imagined hands/feet
subject_id = 22  # carefully cherry-picked to give nice results on such limited data :)
event_codes = [5, 6, 9, 10, 13, 14]
# event_codes = [3,4,5,6,7,8,9,10,11,12,13,14]

# This will download the files if you don't have them yet,
# and then return the paths to the files.
physionet_paths = mne.datasets.eegbci.load_data(subject_id, event_codes)

# Load each of the files
parts = [mne.io.read_raw_edf(path, preload=True, stim_channel='auto',
                             verbose='WARNING')
         for path in physionet_paths]

# Concatenate them
raw = concatenate_raws(parts)

# Find the events in this dataset
events, _ = mne.events_from_annotations(raw)

# Use only EEG channels
eeg_channel_inds = \
    mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Extract trials, only using EEG channels
epoched = mne.Epochs(raw, events, dict(hands_or_left=2, feet_or_right=3),
                     tmin=1, tmax=4.1, proj=False, picks=eeg_channel_inds,
                     baseline=None, preload=True)

##############################################################################
# Convert data to Braindecode format
# ----------------------------------

# Braindecode has a minimalistic ```SignalAndTarget``` class, with
# attributes `X` for the signal and `y` for the labels. `X` should have
# these dimensions: trials x channels x timesteps. `y` should have one
# label per trial.

import numpy as np

# Convert data from volt to millivolt
# Pytorch expects float32 for input and int64 for labels.
X = (epoched.get_data() * 1e6).astype(np.float32)
y = (epoched.events[:, 2] - 2).astype(np.int64)  # 2,3 -> 0,1


# We use the first 40 trials for training and the next 30 trials for
# validation. The validation accuracies can be used to tune hyperparameters
# such as learning rate etc. The final 20 trials are split apart so we have
# a final hold-out evaluation set that is not part of any hyperparameter
# optimization. As mentioned before, this dataset is dangerously small to
# get any meaningful results and only used here for quick demonstration
# purposes.

from braindecode.datautil.signal_target import SignalAndTarget

train_set = SignalAndTarget(X[:40], y=y[:40])
valid_set = SignalAndTarget(X[40:70], y=y[40:70])


##############################################################################
# Create the model
# ----------------

# Braindecode comes with some predefined convolutional neural network
# architectures for raw time-domain EEG. Here, we use the shallow ConvNet
# model from [Deep learning with convolutional neural networks for EEG
# decoding and visualization](https://arxiv.org/abs/1703.05051).

from braindecode.models import ShallowFBCSPNet
from braindecode.torch_ext.util import set_random_seeds  # XXX : move to braindecode.util

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = train_set.X.shape[1]

# final_conv_length = auto ensures we only get a single output in the time dimension
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=train_set.X.shape[2],
                        final_conv_length='auto')
if cuda:
    model.cuda()


# We use [AdamW](https://arxiv.org/abs/1711.05101) to optimize the parameters of our network together with [Cosine Annealing](https://arxiv.org/abs/1608.03983) of the learning rate. We supply some default parameters that we have found to work well for motor decoding, however we strongly encourage you to perform your own hyperparameter optimization using cross validation on your training data.

# <div class="alert alert-info">
#
# We will now use the Braindecode model class directly to perform the training in a few lines of code. If you instead want to use your own training loop, have a look at the [Trialwise Low-Level Tutorial](./TrialWise_LowLevel.html).
#
# </div>

# In[ ]:


# from braindecode.torch_ext.optimizers import AdamW
from torch.optim import Adam
import torch.nn.functional as F
# optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = Adam(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1,)

# ## Run the training

# In[ ]:


n_epochs = 4
model.fit(train_set.X, train_set.y, n_epochs=n_epochs, batch_size=64,
          scheduler='cosine', validation_data=(valid_set.X, valid_set.y))


# The monitored values are also stored into a pandas dataframe:

model.epochs_df

2020-01-14 13:19:39,084 INFO : Run until first stop...
2020-01-14 13:19:39,970 INFO : Epoch 0
2020-01-14 13:19:39,971 INFO : train_loss                3.42033
2020-01-14 13:19:39,971 INFO : valid_loss                2.83930
2020-01-14 13:19:39,972 INFO : train_misclass            0.47500
2020-01-14 13:19:39,973 INFO : valid_misclass            0.46667
2020-01-14 13:19:39,973 INFO : runtime                   0.00000
2020-01-14 13:19:39,974 INFO : 
2020-01-14 13:19:41,138 INFO : Time only for training updates: 1.16s
2020-01-14 13:19:42,007 INFO : Epoch 1
2020-01-14 13:19:42,009 INFO : train_loss                0.80030
2020-01-14 13:19:42,009 INFO : valid_loss                1.41156
2020-01-14 13:19:42,010 INFO : train_misclass            0.40000
2020-01-14 13:19:42,010 INFO : valid_misclass            0.50000
2020-01-14 13:19:42,011 INFO : runtime                   2.05409
2020-01-14 13:19:42,011 INFO : 
2020-01-14 13:19:43,128 INFO : Time only for training updates: 1.12s
2020-01-14 13:1

,train_loss,valid_loss,train_misclass,valid_misclass,runtime
0,3.420333,2.839304,0.475,0.466667,0.000000
1,0.800297,1.411565,0.400,0.500000,2.054093
2,0.363404,0.988987,0.150,0.366667,1.990847
3,0.257427,0.825735,0.125,0.366667,1.886488
4,0.189247,0.681307,0.100,0.333333,1.985432
